# **Importation**

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
# Accéder au drive
drive.mount('/content/drive')

Mounted at /content/drive


# **TRANSFORMATION COULEUR**

In [16]:
df_palette_hexa = pd.read_csv('/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/ML/df_palette_ML.csv')
df_palette_hexa.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/ML/df_palette_ML.csv'

In [ ]:
from PIL import ImageColor
df_palette_RGB = pd.DataFrame(data = {'couleur_peau' : [],  'style' : [], 'C1' : [], 'C2' : [], 'C3' : []})

df_palette_RGB['couleur_peau'] = df_palette_hexa['couleur_peau'].apply(lambda x: ImageColor.getcolor(x, "RGB"))
df_palette_RGB['style'] = df_palette_hexa['style']
df_palette_RGB['C1'] = df_palette_hexa['C1'].apply(lambda x: ImageColor.getcolor(x, "RGB"))
df_palette_RGB['C2'] = df_palette_hexa['C2'].apply(lambda x: ImageColor.getcolor(x, "RGB"))
df_palette_RGB['C3'] = df_palette_hexa['C3'].apply(lambda x: ImageColor.getcolor(x, "RGB"))
df_palette_RGB.head()

In [ ]:
df_palette_RGB.to_csv("/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/data_palette.csv", index = False)

# **TRAITEMENT DATASET PRODUIT**

In [ ]:
df_produit = pd.read_csv('/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/data_merged.csv')
print(df_produit.shape)
df_produit.head()

In [ ]:
# DICTIONNAIRE COULEUR --> RGB
teinte_code = {'claire' : ['#FCF5E8','#FAF2E7','#FAF1D6'], 'medium' : ['#EEDABA','#D7B98B','#AB814E'],
               'foncee' : ['#996748','#754736','#4E3830','#3F322C'], 'universelle' : ['#EEDABA','#D7B98B','#AB814E']}
teinte_code_rgb = {}
for key, value in teinte_code.items():
    teinte_code_rgb[key] = [ImageColor.getcolor(value[0], "RGB"), ImageColor.getcolor(value[1], "RGB"), ImageColor.getcolor(value[2], "RGB")]

couleur_code_rgb = {"metallique" : (128,128,128),"rouge" : (255,0,0),'transparent' : (245,255,250),
                'bleu': (0,0,255), 'orange' : (255,69,0), 'noir': (0,0,0), 'gris_argent' : (192,192,192),
                'brun_cuivre' : (179, 103, 0), 'beige' : (200,173,127), 'violet' : (238,130,238), 'rose' : (255,192,203),
                'jaune_or' : (239, 216, 7), 'multi' : (255,255,255)}

In [ ]:
# CREATION COLONNE RGB
import random
for i in range(len(df_produit)):
    if (str(df_produit.loc[i,'teint']) == 'nan') == False:
        teinte = df_produit.loc[i,'teint']
        code = random.choice(teinte_code_rgb[teinte])
        df_produit.loc[i,'RGB'] = str(code)
    elif (str(df_produit.loc[i,'color']) == 'nan') == False:
        df_produit.loc[i,'RGB'] = str(couleur_code_rgb[df_produit.loc[i,'color']])


# On complète les nan par transparent : 'transparent' : (245,255,250)
df_produit['RGB'].fillna('(245,255,250)', inplace = True)

df_produit

In [ ]:
# CREATION COLONNES R, G et B
df_produit['R'] = df_produit['RGB'].apply(lambda x : int(x.split(',')[0][1:]))
df_produit['G'] = df_produit['RGB'].apply(lambda x : int(x.split(',')[1]))
df_produit['B'] = df_produit['RGB'].apply(lambda x : int(x.split(',')[2][:-1]))
df_produit.head()

In [ ]:
df_produit.to_csv("/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/data_to_ML.csv", index = False)

In [ ]:
df_produit['category'].unique()

# **MACHINE LEARNING**

**MODE 1 : l'utilisateur n'a pas de produit**

In [17]:
df_palette_RGB['style'].unique()

NameError: name 'df_palette_RGB' is not defined

In [15]:
df_data_ML = pd.read_csv("/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/data_to_ML.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/WildCodeSchool/hackathon/Hackathon2/Dataset_webscrapped/data_to_ML.csv'

In [13]:
# ENTREE : COULEUR DE PEAU      ET      STYLE
couleur_de_peau = (117, 71, 54)
style = "complement"

# ROUGE A LEVRE
df_levre = df_data_ML[df_data_ML['category'] == 'levres']
couleur_levre = df_palette_RGB[(df_palette_RGB['couleur_peau'] == couleur_de_peau)&(df_palette_RGB['style'] == style)]['C1'].values[0]
couleur_levre
# ML
from sklearn.neighbors import NearestNeighbors
X = df_levre[['R','G','B']]
KNN = NearestNeighbors(n_neighbors=1).fit(X)
rouge_levre = KNN.kneighbors([list(couleur_levre)])
print("On vous propose le rouge à lèvre :")
print(df_levre.iloc[rouge_levre[1][0][0]])

# FARD A PAUPIERE
df_fard = df_data_ML[df_data_ML['category'] == 'fard-a-paupieres-palette']
couleur_fard = df_palette_RGB[(df_palette_RGB['couleur_peau'] == couleur_de_peau)&(df_palette_RGB['style'] == style)]['C2'].values[0]
print(couleur_fard)
couleur_fard
# ML
X = df_fard[['R','G','B']]
KNN = NearestNeighbors(n_neighbors=1).fit(X)
rouge_levre = KNN.kneighbors([list(couleur_fard)])
print("On vous propose le fard à paupière :")
print(df_fard.iloc[rouge_levre[1][0][0]])

# FOND DE TEINT
print('On vous propose le fond de teint :')
df_fond_teint = df_data_ML[df_data_ML['category'] == 'teint']
print(df_fond_teint[df_fond_teint['RGB'] == str(couleur_de_peau)].iloc[0])

NameError: name 'df_data_ML' is not defined

**MODE 2 : l'utilisateur a un produit :**
- fond de teint : ne change rien
- rouge à lèvre
- fard à paupière

In [12]:
# ENTREE : COULEUR DE PEAU      ET      STYLE          ET           PRODUIT
couleur_de_peau = (250, 241, 214) #
style = "complement"
couleur_levre = (255,0,0) # rouge
couleur_fard = (255,0,0) # rouge


from sklearn.neighbors import NearestNeighbors
# ROUGE A LEVRE
df_levre = df_produit[df_produit['category'] == 'levres']
couleur_levre = (255,0,0) # rouge
# ML
X = df_levre[['R','G','B']]
KNN = NearestNeighbors(n_neighbors=1).fit(X)
rouge_levre = KNN.kneighbors([list(couleur_levre)])
print("On vous propose le rouge à lèvre :")
print(df_levre.iloc[rouge_levre[1][0][0]])

# FARD A PAUPIERE
df_fard = df_produit[df_produit['category'] == 'fard-a-paupieres-palette']
couleur_fard = (255,0,0) # rouge
# ML
X = df_fard[['R','G','B']]
KNN = NearestNeighbors(n_neighbors=1).fit(X)
rouge_levre = KNN.kneighbors([list(couleur_fard)])
print("On vous propose le fard à paupière :")
print(df_fard.iloc[rouge_levre[1][0][0]])

# FOND DE TEINT
print('On vous propose le fond de teint :')
df_fond_teint = df_produit[df_produit['category'] == 'teint']
print(df_fond_teint[df_fond_teint['RGB'] == str(couleur_de_peau)].iloc[0])



NameError: name 'df_produit' is not defined

**Récupération couleur peau**

In [4]:
    # if ita <= 10:
    #     return "dark"
    # elif 10 < ita <= 19:
    #     return "tan1"
    # elif 19 < ita <= 28:
    #     return "tan2"
    # elif 28 < ita <= 34.5:
    #     return "int1"
    # elif 34.5 < ita <= 41:
    #     return "int2"
    # elif 41 < ita <= 48:
    #     return "lt1"
    # elif 48 < ita <= 55:
    #     return "lt2"
    # elif 55 < ita:
    #     return "very_lt"
    # else:
    #     print(f"None cat: {ita}")


dico_color = {"dark" : (169, 117, 78),"tan1" : (182, 137, 96),"tan2" : (196, 151, 109),
              "int1" : (212, 174, 129),"int2" : (217, 179, 134) ,"lt1" : (219, 181, 136),
              "lt2" : (222, 183, 140),"very_lt" : (225,191,154)}

In [ ]:
# "very_lt" : (254, 245, 238)
# "lt2" : (249, 236, 220)
#  "lt1" : (247, 228, 196)
# "int2" : (210, 181, 147)
# "int1" : (167, 123, 78)
# "tan2" : (153, 104, 72)
# "tan1" : (83, 60, 52)
# "dark" : (53, 44, 37)
#
#

In [9]:
for key in dico_color.keys():
    print(key, end = ' : ')
    file = f"{key}.jpg"
    whole_image_ita = get_ita(image=Image.open(file))
    kinyanjui_type = get_kinyanjui_type(whole_image_ita)
    print(kinyanjui_type)

dark : 

FileNotFoundError: [Errno 2] No such file or directory: 'dark.jpg'

In [6]:
!pip install derm_ita

In [20]:
df = pd.read_csv("/content/mst-e_image_details.csv")

In [21]:
df

,image_ID,pose,lighting,mask,subject_name,MST
0,PXL_20220922_183640936.PORTRAIT.jpg,frontal,poorly_lit,0,subject_18,1
1,PXL_20220922_183639710.PORTRAIT.jpg,bottom,poorly_lit,0,subject_18,1
2,PXL_20220922_183638393.PORTRAIT.jpg,side,poorly_lit,0,subject_18,1
3,PXL_20220922_183636542.PORTRAIT.jpg,side,poorly_lit,0,subject_18,1
4,PXL_20220922_183634690.PORTRAIT.jpg,side,poorly_lit,0,subject_18,1
...,...,...,...,...,...,...
1541,PXL_20220922_132959831.PORTRAIT.jpg,facing_camera,well_lit,0,subject_0,3
1542,PXL_20220922_132914899.PORTRAIT.jpg,facing_camera,well_lit,0,subject_0,3
1543,PXL_20220922_132911049.PORTRAIT.jpg,bottom,well_lit,0,subject_0,3
1544,PXL_20220922_132903030.PORTRAIT.jpg,side,well_lit,0,subject_0,3


In [22]:
df.columns

Index(['image_ID', 'pose', 'lighting', 'mask', 'subject_name', 'MST'], dtype='object')

In [7]:
from derm_ita import get_ita
from PIL import Image
from derm_ita import get_kinyanjui_type

In [35]:
whole_image_ita = get_ita(image=Image.open('/content/MST_2_JPEG.jpg'))
kinyanjui_type = get_kinyanjui_type(whole_image_ita)
kinyanjui_type

'very_lt'

In [37]:
whole_image_ita = get_ita(image=Image.open('/content/MST_9_JPEG.jpg'))
kinyanjui_type = get_kinyanjui_type(whole_image_ita)
kinyanjui_type

'dark'

In [36]:
whole_image_ita = get_ita(image=Image.open('/content/MST_10_JPEG.jpg'))
kinyanjui_type = get_kinyanjui_type(whole_image_ita)
kinyanjui_type

'dark'

# **TEST MACHINE LEARNING**

In [ ]:
df_palette = pd.DataFrame(data = {'couleur_peau' : [],  'style' : [], 'couleur_levre' : [], 'couleur_teint' : [], 'couleur_blush' : []})
df_palette

In [ ]:
df_couleur = pd.DataFrame(data = {'id' : [0,1,2,3,4],  'R' : [],  'G' : [],  'B' : []})

In [ ]:
data_color =[[249, 235, 234], [230, 176, 170], [162, 217, 206],
            [242, 243, 244], [93, 109, 126]]

df_couleur = pd.DataFrame(data = {'id' : [],  'R' : [],  'G' : [],  'B' : []})
for i in range(len(data_color)):
    df_temp = pd.DataFrame(data = {'id' : [i],  'R' : [data_color[i][0]],  'G' : [data_color[i][1]],  'B' : [data_color[i][2]]})
    df_couleur = df_couleur.append(df_temp, ignore_index=True)

df_couleur

In [ ]:
data_color =[[249, 235, 234 ,'rouge clair'], [146, 43, 33, 'rouge foncé'], [244, 236, 247, 'violet clair'],
            [108, 52, 131, 'violet foncé'], [93, 173, 226, 'bleu'], [208, 236, 231, 'vert clair'],
            [30, 132, 73, 'vert foncé']]

df_couleur = pd.DataFrame(data = {'id' : [],  'R' : [],  'G' : [],  'B' : [],  'color' : []})
for i in range(len(data_color)):
    df_temp = pd.DataFrame(data = {'id' : [i],  'R' : [data_color[i][0]],  'G' : [data_color[i][1]],  'B' : [data_color[i][2]], 'color' : [data_color[i][3]]})
    df_couleur = df_couleur.append(df_temp, ignore_index=True)


X = df_couleur[['R','G','B']]
######## On retire la valeur recherchée
couleur_base = [244, 236, 247]
if len(X[(X['R'] == couleur_base[0])&(X['G'] == couleur_base[1])&(X['B'] == couleur_base[2])]) > 0:
    X.drop(X[(X['R'] == couleur_base[0])&(X['G'] == couleur_base[1])&(X['B'] == couleur_base[2])].index, inplace = True)
    print(f"Suppression {couleur_base}")
df_couleur

In [ ]:
from sklearn.neighbors import NearestNeighbors

KNN = NearestNeighbors(n_neighbors=1).fit(X)
close_color = KNN.kneighbors([couleur_base])


print('La distance est : ',close_color[0][0][0])
print(df_couleur.loc[df_couleur.iloc[close_color[1][0][0]][0]])

In [ ]:
produit_possede = input("")